In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
# import quant_dorefa
import model

In [2]:
BATCH_SIZE=512
test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=BATCH_SIZE, shuffle=True)

In [3]:
# for data, target in test_loader:
#     print(data.size())
# a = test_loader.data()
a = list(test_loader)
test_one_data = a[0][0][0]
# print(test_loader[0])
    

In [11]:
import numpy as np
print(test_one_data)
test_one_data.numpy().tofile('test_data.bin')

tensor([[[[-0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
           -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
           -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
           -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
           -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
           -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
           -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
           -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
           -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
           -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
           -0.4242, -0.4242, -0.424

           -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242]]]])


In [5]:
model = model.ConvNet()
model.load_state_dict(torch.load("model.pkl", map_location='cpu'))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [6]:
from matplotlib import pyplot as plt
import numpy as np
a = test_one_data.numpy()
a = a.reshape(28, 28)
plt.imshow(a)
plt.show()

<Figure size 640x480 with 1 Axes>

In [7]:
test_one_data = test_one_data.reshape(1, 1, 28, 28)
print(test_one_data)
res = model(test_one_data)
print(res)  # 结果是1


tensor([[[[-0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
           -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
           -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
           -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
           -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
           -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
           -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
           -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
           -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
           -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
           -0.4242, -0.4242, -0.424

           -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242]]]])
tensor([[-28.2405, -25.2959, -19.4056, -22.8052, -29.1311, -26.7797, -54.1954,
           0.0000, -27.4390, -14.0282]], grad_fn=<LogSoftmaxBackward>)


In [33]:
print(model.classifier)
# 提取不同层输出的 主要代码
class LayerActivations:
    features = None
    def __init__(self, model, layer_num):
        self.hook = model[layer_num].register_forward_hook(self.hook_fn)
    def hook_fn(self, module, input, output):
        self.features = output.cpu()
    def remove(self):
        self.hook.remove()
 
 
conv_out = LayerActivations(model.classifier, 2)  # 提出第 一个卷积层的输出 
print(model(test_one_data))
conv_out.remove()  #
act = conv_out.features  

print(act[0])


Sequential(
  (0): Linear(in_features=6272, out_features=128, bias=True)
  (1): ReLU(inplace)
  (2): Linear(in_features=128, out_features=10, bias=True)
)
tensor([[-28.2405, -25.2959, -19.4056, -22.8052, -29.1311, -26.7797, -54.1954,
           0.0000, -27.4390, -14.0282]], grad_fn=<LogSoftmaxBackward>)
tensor([ -6.4658,  -3.5212,   2.3691,  -1.0305,  -7.3564,  -5.0050, -32.4207,
         21.7747,  -5.6642,   7.7465], grad_fn=<SelectBackward>)


In [9]:
a = nn.Conv2d(1, 16, kernel_size=(3,3), bias=False)
print(type(None))

<class 'NoneType'>
